<a href="https://colab.research.google.com/github/wendirad/weekTwo/blob/task_4/notebooks/task_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import  pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans
from mpl_toolkits.mplot3d import Axes3D
from sklearn.preprocessing import StandardScaler
from scipy.spatial.distance import euclidean
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [ ]:
#read xlssx data
data = pd.read_excel('data.xlsx')
data.head()

,Bearer Id,Start,Start ms,End,End ms,Dur. (ms),IMSI,MSISDN/Number,IMEI,Last Location Name,...,Youtube DL (Bytes),Youtube UL (Bytes),Netflix DL (Bytes),Netflix UL (Bytes),Gaming DL (Bytes),Gaming UL (Bytes),Other DL (Bytes),Other UL (Bytes),Total UL (Bytes),Total DL (Bytes)
0,13114483460844900352,2019-04-04 12:01:18,770.0,2019-04-25 14:35:31,662.0,1823652.0,2.082014e+14,3.366496e+10,3.552121e+13,9164566995485190,...,15854611.0,2501332.0,8198936.0,9656251.0,278082303.0,14344150.0,171744450.0,8814393.0,36749741.0,308879636.0
1,13114483482878900224,2019-04-09 13:04:04,235.0,2019-04-25 08:15:48,606.0,1365104.0,2.082019e+14,3.368185e+10,3.579401e+13,L77566A,...,20247395.0,19111729.0,18338413.0,17227132.0,608750074.0,1170709.0,526904238.0,15055145.0,53800391.0,653384965.0
2,13114483484080500736,2019-04-09 17:42:11,1.0,2019-04-25 11:58:13,652.0,1361762.0,2.082003e+14,3.376063e+10,3.528151e+13,D42335A,...,19725661.0,14699576.0,17587794.0,6163408.0,229584621.0,395630.0,410692588.0,4215763.0,27883638.0,279807335.0
3,13114483485442799616,2019-04-10 00:31:25,486.0,2019-04-25 07:36:35,171.0,1321509.0,2.082014e+14,3.375034e+10,3.535661e+13,T21824A,...,21388122.0,15146643.0,13994646.0,1097942.0,799538153.0,10849722.0,749039933.0,12797283.0,43324218.0,846028530.0
4,13114483499480700928,2019-04-12 20:10:23,565.0,2019-04-25 10:40:32,954.0,1089009.0,2.082014e+14,3.369980e+10,3.540701e+13,D88865A,...,15259380.0,18962873.0,17124581.0,415218.0,527707248.0,3529801.0,550709500.0,13910322.0,38542814.0,569138589.0


In [ ]:
len(data)

150001

In [ ]:
customer_metrics = data.groupby('MSISDN/Number').agg(
    session_frequency=('Bearer Id', 'count'),
    total_duration_ms=('Dur. (ms)', 'sum'),
    total_download=('Total DL (Bytes)', 'sum'),
    total_upload=('Total UL (Bytes)', 'sum')
).reset_index()


In [ ]:
customer_metrics['total_traffic'] = customer_metrics['total_download'] + customer_metrics['total_upload']


In [ ]:

top_session_frequency = customer_metrics.nlargest(10, 'session_frequency')
top_total_duration = customer_metrics.nlargest(10, 'total_duration_ms')
top_total_traffic = customer_metrics.nlargest(10, 'total_traffic')


In [ ]:

top_session_frequency,top_total_duration,top_total_traffic

(       MSISDN/Number  session_frequency  total_duration_ms  total_download  \
 13526   3.362632e+10                 18          8791927.0    7.301517e+09   
 6437    3.361489e+10                 17          9966898.0    8.156743e+09   
 13180   3.362578e+10                 17         18553754.0    7.770043e+09   
 37052   3.365973e+10                 16          4035428.0    7.081602e+09   
 76363   3.367588e+10                 15          4865947.0    7.309542e+09   
 92923   3.376054e+10                 15          9279434.0    7.811295e+09   
 65118   3.366716e+10                 13          8744914.0    5.052068e+09   
 666     3.360313e+10                 12          6287761.0    4.540607e+09   
 1279    3.360452e+10                 12          5207990.0    5.096079e+09   
 13994   3.362708e+10                 12          4703516.0    5.309479e+09   
 
        total_upload  total_traffic  
 13526   669650721.0   7.971167e+09  
 6437    689483001.0   8.846226e+09  
 13180   729577

In [ ]:
scaler = MinMaxScaler()


In [ ]:

normalized_metrics = scaler.fit_transform(
    customer_metrics[['session_frequency', 'total_duration_ms', 'total_traffic']]
)

In [ ]:

kmeans = KMeans(n_clusters=3, random_state=42)
customer_metrics['engagement_cluster_norm'] = kmeans.fit_predict(normalized_metrics)

In [ ]:
customer_metrics[['MSISDN/Number', 'session_frequency', 'total_duration_ms', 'total_traffic', 'engagement_cluster_norm']]


,MSISDN/Number,session_frequency,total_duration_ms,total_traffic,engagement_cluster_norm
0,3.360100e+10,1,116720.0,8.786906e+08,2
1,3.360100e+10,1,181230.0,1.568596e+08,2
2,3.360100e+10,1,134969.0,5.959665e+08,2
3,3.360101e+10,1,49878.0,4.223207e+08,2
4,3.360101e+10,2,37104.0,1.457411e+09,0
...,...,...,...,...,...
106851,3.379000e+10,1,8810.0,7.146416e+08,2
106852,3.379000e+10,1,140988.0,4.803073e+08,2
106853,3.197021e+12,1,877385.0,2.321240e+08,2
106854,3.370000e+14,1,253030.0,5.962878e+08,2


In [ ]:
centroids = kmeans.cluster_centers_
f"Cluster centroids: {centroids}"

'Cluster centroids: [[0.06915313 0.01112472 0.12386147]\n [0.19091316 0.03029614 0.25485257]\n [0.00247487 0.00538298 0.05244953]]'

In [ ]:
aggregated_data = df.groupby('MSISDN/Number').agg({
    'TCP Retransmission (Bytes)': 'mean',  # Average TCP Retransmission
    'RTT (ms)': 'mean',                   # Average RTT
    'Throughput (kbps)': 'mean',          # Average Throughput
    'Handset Type': lambda x: x.mode()[0] # Most common handset type
}).reset_index()

# Rename columns for clarity
aggregated_data.rename(columns={
    'TCP Retransmission (Bytes)': 'Average TCP Retransmission (Bytes)',
    'RTT (ms)': 'Average RTT (ms)',
    'Throughput (kbps)': 'Average Throughput (kbps)',
    'Handset Type': 'Most Common Handset Type'
}, inplace=True)

print(aggregated_data)

In [ ]:
# Engagement and experience metrics columns
engagement_columns = ['Total UL (Bytes)', 'Total DL (Bytes)', 'Dur. (ms)']  # Replace with actual engagement-related columns
experience_columns = ['TCP DL Retrans. Vol (Bytes)','TCP UL Retrans. Vol (Bytes)','Avg RTT DL (ms)','Avg RTT UL (ms)', 'Avg Bearer TP DL (kbps)']  # Replace with actual experience-related columns

# Replace with cluster centroids obtained from clustering
# Ensure that the cluster centroids have the correct number of dimensions
# The less_engaged_cluster should have 3 elements to match engagement_columns
less_engaged_cluster = [10000, 50000, 20000]  # Example values for less engaged cluster center

# The worst_experience_cluster should have 5 elements to match experience_columns
worst_experience_cluster = [500, 400, 50, 60, 70]    # Example values for worst experience cluster center # Added 2 more elements to match the shape of experience_columns


# Before calculating the scores, replace infs and NaNs with finite values
# For example, replace with the column mean:
for col in engagement_columns + experience_columns:
    data[col] = data[col].fillna(data[col].mean())
    data[col] = data[col].replace([np.inf, -np.inf], data[col].mean()) # this was in addition of the previous line


# Calculate Engagement Score (Euclidean distance to less engaged cluster)
data['Engagement Score'] = data[engagement_columns].apply(
    lambda row: euclidean(row, less_engaged_cluster), axis=1
)

# Calculate Experience Score (Euclidean distance to worst experience cluster)
data['Experience Score'] = data[experience_columns].apply(
    lambda row: euclidean(row, worst_experience_cluster), axis=1
)

# Output a preview of the dataset with the scores
print(data[['Engagement Score', 'Experience Score']].head())

   Engagement Score  Experience Score
0      3.110125e+08      2.082326e+07
1      6.555469e+08      2.082326e+07
2      2.811457e+08      2.082326e+07
3      8.470877e+08      2.082326e+07
4      5.703926e+08      2.082326e+07


In [ ]:
# prompt: Consider the average of both engagement & experience scores as  the satisfaction score & report the top 10 satisfied customer

# Calculate Satisfaction Score
data['Satisfaction Score'] = (data['Engagement Score'] + data['Experience Score']) / 2

# Report the top 10 satisfied customers
top_10_satisfied = data.nlargest(10, 'Satisfaction Score')
print(top_10_satisfied[['MSISDN/Number', 'Satisfaction Score']])

        MSISDN/Number  Satisfaction Score
67481    3.366613e+10        2.542339e+09
41182    3.365863e+10        2.513163e+09
122572   3.365908e+10        2.437290e+09
77950    3.376211e+10        2.433802e+09
65135             NaN        2.417919e+09
39608    3.366491e+10        2.417400e+09
34636    3.366045e+10        2.388037e+09
119416   3.366682e+10        2.384575e+09
3741     3.365871e+10        2.375074e+09
78810    3.366486e+10        2.371530e+09


In [ ]:

X = data[['Engagement Score', 'Experience Score']]  # Features
y = data['Satisfaction Score']  # Target variable

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize and train a linear regression model
model = LinearRegression()
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")

Mean Squared Error: 1.557481445495815e-14


In [ ]:
# K-means clustering
kmeans = KMeans(n_clusters=2, random_state=42)
data['Cluster'] = kmeans.fit_predict(data[['Engagement Score', 'Experience Score']])

# Print cluster assignments
print(data[['MSISDN/Number', 'Engagement Score', 'Experience Score', 'Cluster']])

        MSISDN/Number  Engagement Score  Experience Score  Cluster
0        3.366496e+10      3.110125e+08      2.082326e+07        1
1        3.368185e+10      6.555469e+08      2.082326e+07        0
2        3.376063e+10      2.811457e+08      2.082326e+07        1
3        3.375034e+10      8.470877e+08      2.082326e+07        0
4        3.369980e+10      5.703926e+08      2.082326e+07        0
...               ...               ...               ...      ...
149996   3.365069e+10      5.770093e+08      2.082326e+07        0
149997   3.366345e+10      6.677461e+08      2.082326e+07        0
149998   3.362189e+10      5.937631e+08      2.082326e+07        0
149999   3.361962e+10      3.730235e+08      2.082326e+07        1
150000            NaN      4.564486e+08      2.082327e+07        1

[150001 rows x 4 columns]


In [ ]:
# Aggregate the average satisfaction and experience score per cluster
cluster_summary = data.groupby('Cluster').agg(
    {'Satisfaction Score': 'mean', 'Experience Score': 'mean'}
).reset_index()

cluster_summary

,Cluster,Satisfaction Score,Experience Score
0,0,3.439554e+08,2.091568e+07
1,1,1.346683e+08,2.118835e+07


from matplotlib import pyplot as plt
cluster_summary['Cluster'].plot(kind='hist', bins=20, title='Cluster')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
cluster_summary['Satisfaction Score'].plot(kind='hist', bins=20, title='Satisfaction Score')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
cluster_summary['Experience Score'].plot(kind='hist', bins=20, title='Experience Score')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
cluster_summary.plot(kind='scatter', x='Cluster', y='Satisfaction Score', s=32, alpha=.8)
plt.gca().spines[['top', 'right',]].set_visible(False)

...

from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  palette = list(sns.palettes.mpl_palette('Dark2'))
  xs = series['Experience Score']
  ys = series['Satisfaction Score']
  
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = cluster_summary.sort_values('Experience Score', ascending=True)
_plot_series(df_sorted, '')
sns.despine(fig=fig, ax=ax)
plt.xlabel('Experience Score')
_ = plt.ylabel('Satisfaction Score')

from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  palette = list(sns.palettes.mpl_palette('Dark2'))
  counted = (series['Experience Score']
                .value_counts()
              .reset_index(name='counts')
              .rename({'index': 'Experience Score'}, axis=1)
              .sort_values('Experience Score', ascending=True))
  xs = counted['Experience Score']
  ys = counted['counts']
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = cluster_summary.sort_values('Experience Score', ascending=True)
_plot_series(df_sorted, '')
sns.despine(fig=fig, ax=ax)
plt.xlabel('Experience Score')
_ = plt.ylabel('count()')

from matplotlib import pyplot as plt
cluster_summary['Cluster'].plot(kind='line', figsize=(8, 4), title='Cluster')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
cluster_summary['Satisfaction Score'].plot(kind='line', figsize=(8, 4), title='Satisfaction Score')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
cluster_summary['Experience Score'].plot(kind='line', figsize=(8, 4), title='Experience Score')
plt.gca().spines[['top', 'right']].set_visible(False)